In [1]:
import os, sys
from dotenv import load_dotenv

#load_dotenv(override=True)
load_dotenv("/mnt/d/agent_research/github-repos/home/AAWS_project/.env", override=True)

# PROJECT_ROOT를 .env에서 읽기 (없으면 현재 디렉토리)
project_root = os.getenv("PROJECT_ROOT", os.getcwd())

# 프로젝트 루트가 유효하지 않으면, 현재 위치에서 상위로 찾기
if not os.path.exists(os.path.join(project_root, "app")):
    # 상위 디렉토리 탐색
    current = os.getcwd()
    for _ in range(5):
        if os.path.exists(os.path.join(current, "app")):
            project_root = current
            break
        current = os.path.dirname(current)

# Working Directory 설정
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import app

print(f"✅ Project Root: {project_root}")
print(f"✅ Working Directory: {os.getcwd()}")

✅ Project Root: /mnt/d/agent_research/github-repos/home/AAWS_project
✅ Working Directory: /mnt/d/agent_research/github-repos/home/AAWS_project


In [2]:
import asyncio
from browser_use import Agent, Browser, ChatGoogle

import nest_asyncio

# 주피터 노트북 환경에서 비동기 루프 중복 실행 오류를 방지하기 위해 사용합니다.
nest_asyncio.apply()

# 가상 디스플레이(noVNC) 번호를 지정합니다.
# 이 설정이 있어야 GUI 브라우저가 가상 화면(:1)에 나타납니다.
# os.environ["DISPLAY"] = ":1"

/home/hukim/env_langchain_123/lib/python3.12/site-packages/requests/__init__.py:113: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (6.0.0.post1)/charset_normalizer (3.4.4) doesn't match a supported version!
  warnings.warn(


# 동적 N계층 Blueprint 스키마 설계

핵심 아이디어: **"계층을 레고 블록처럼, 필요한 만큼만 쌓는다"**

---

## 클래스 구조

NavigatorBlueprintCollection ← Navigator의 최종 출력 (1개 이상의 Blueprint 묶음) 

└── blueprints: list[NavigatorBlueprint] 

└── layers: list[PageLayer] ← 탐색 계층 하나를 표현하는 단위 블록



##### `PageLayer` - 탐색 계층 단위 블록

| 필드 | 설명 |
|------|------|
| `layer_name` | 이 계층의 역할 이름 (예: '기사 목록', '상품 상세') |
| `url_pattern` | 이 계층의 URL 구조 예시 |
| `selectors` | `{필드명: CSS셀렉터}` 딕셔너리 |
| `navigate_to_next` | 다음 계층 링크의 CSS 셀렉터. **마지막 계층이면 반드시 `None`** |
| `pagination_method` | 페이지네이션 방식 (URL파라미터 / AJAX버튼 / 무한스크롤 / `None`) |

**`navigate_to_next` 필드가 핵심입니다:**
- 값이 있으면 → 그 CSS 셀렉터로 링크를 추출하여 다음 계층으로 이동
- `None`이면 → 이 계층이 마지막 (데이터 수집 완료)

##### `NavigatorBlueprint` - 단일 수집 작업 설계도

| 필드 | 설명 |
|------|------|
| `entry_urls` | 크롤링 시작 URL 목록. 구조가 같고 시작점만 다른 경우 여러 개 지정 |
| `total_layers` | 탐색에 필요한 총 계층 수 (layers 리스트 길이와 동일) |
| `layers` | 탐색 순서대로 정렬된 `PageLayer` 목록 |
| `rendering_type` | `"Static SSR"` 또는 `"Dynamic CSR/JS"` |
| `anti_bot_notes` | 로그인, 팝업, 캡차 등 주의사항. 없으면 `"없음"` |

##### `NavigatorBlueprintCollection` - Navigator의 최종 출력

| 필드 | 설명 |
|------|------|
| `total_jobs` | 총 Blueprint 수 |
| `blueprints` | Blueprint 목록 |

**Blueprint 개수 결정 기준:**
- 구조(계층/셀렉터)가 같고 시작 URL만 다른 경우 → **Blueprint 1개**, `entry_urls`에 복수 URL
- 구조가 근본적으로 다른 경우 (다른 사이트 등) → **Blueprint 별도 생성**

---

## 스키마 검증 예시 3가지

| 예시 | 설명 |
|------|------|
| 예시 1 | Blueprint 1개, `entry_urls` 1개 (단일 섹션) |
| 예시 2 | Blueprint 1개, `entry_urls` 2개 (구조 동일, URL만 다름) |
| 예시 3 | Blueprint 2개 (사이트 구조가 달라 분리) |


In [ ]:
import json
from typing import Optional
from pydantic import BaseModel, Field, field_validator

# ==========================================
# 동적 N계층 Blueprint 스키마
# ==========================================

class PageLayer(BaseModel):
    """하나의 탐색 계층을 표현하는 단위 블록"""
    layer_name: str = Field(
        description="이 계층의 역할 이름 (예: '기사 목록', '상품 상세')"
    )
    url_pattern: str = Field(
        description="이 계층의 URL 구조 예시 또는 진입점 URL (실제 시작 URL은 entry_urls 참조)"
    )
    selectors: dict[str, str] = Field(
        description="이 계층에서 수집할 데이터의 CSS 셀렉터 딕셔너리 (key: 필드명, value: CSS 셀렉터)"
    )
    navigate_to_next: Optional[str] = Field(
        default=None,
        description="다음 계층으로 이동하는 링크의 CSS 셀렉터. 마지막 계층이면 반드시 None."
    )
    pagination_method: Optional[str] = Field(
        default=None,
        description="페이지네이션 방식 (URL파라미터 / AJAX버튼 / 무한스크롤 / None)"
    )

    @field_validator("selectors", mode="before")
    @classmethod
    def parse_selectors(cls, v):
        if isinstance(v, str):
            try:
                return json.loads(v)
            except Exception:
                pass
        return v

    # ↓ 이 validator 추가
    @field_validator("navigate_to_next", "pagination_method", mode="before")
    @classmethod
    def parse_none_string(cls, v):
        """LLM이 None을 문자열 "None"으로 반환하는 경우를 처리합니다."""
        if v in ("None", "null", "없음", "N/A", ""):
            return None
        return v

class NavigatorBlueprint(BaseModel):
    """Navigator가 Coder에게 전달하는 동적 N계층 크롤링 설계 도면 (단일 구조)"""
    entry_urls: list[str] = Field(
        description=(
            "크롤링을 시작할 URL 목록. "
            "구조(계층/셀렉터)가 동일하고 시작점만 다른 경우 여러 개 지정. "
            "예) 정치 섹션 URL + 사회 섹션 URL"
        )
    )
    total_layers: int = Field(
        description="탐색에 필요한 총 계층 수 (layers 리스트의 길이와 동일)"
    )
    layers: list[PageLayer] = Field(
        description="탐색 순서대로 정렬된 PageLayer 목록. layers[0]은 entry_urls 각각에 반복 적용됨."
    )
    rendering_type: str = Field(
        description="Static SSR 또는 Dynamic CSR/JS"
    )
    anti_bot_notes: str = Field(
        description="로그인 필요 여부, 팝업, 캡차, 우회 조언 등. 없으면 '없음'"
    )


class NavigatorBlueprintCollection(BaseModel):
    """Navigator가 반환하는 Blueprint 모음 (1개 이상)"""
    total_jobs: int = Field(
        description="총 Blueprint 수. 구조가 같으면 1개, 구조가 다른 사이트/섹션은 각각 1개."
    )
    blueprints: list[NavigatorBlueprint] = Field(
        description=(
            "수집 작업별 Blueprint 목록. "
            "- 구조 동일 + 시작 URL만 다름 → Blueprint 1개, entry_urls에 복수 URL "
            "- 구조가 근본적으로 다름 → Blueprint를 별도 생성하여 복수 반환"
        )
    )


# ==========================================
# ✅ 스키마 검증 예시
# ==========================================

# 예시 1: 단일 섹션 (네이버 금융 주식 시세) - Blueprint 1개, entry_urls 1개
sample_single = NavigatorBlueprintCollection(
    total_jobs=1,
    blueprints=[
        NavigatorBlueprint(
            entry_urls=["https://finance.naver.com/sise/sise_rise.nhn"],
            total_layers=1,
            layers=[
                PageLayer(
                    layer_name="주식 상승 종목 테이블",
                    url_pattern="https://finance.naver.com/sise/sise_rise.nhn",
                    selectors={"stock_name": "td.col_type1 a", "change_rate": "td.rate_t"},
                    navigate_to_next=None,
                    pagination_method="URL파라미터"
                )
            ],
            rendering_type="Static SSR",
            anti_bot_notes="없음"
        )
    ]
)

# 예시 2: 동일 구조, 출발 URL만 다른 경우 (정치면 + 사회면) - Blueprint 1개, entry_urls 2개
sample_multi_url = NavigatorBlueprintCollection(
    total_jobs=1,
    blueprints=[
        NavigatorBlueprint(
            entry_urls=[
                "https://news.naver.com/section/100",  # 정치
                "https://news.naver.com/section/102",  # 사회
            ],
            total_layers=2,
            layers=[
                PageLayer(
                    layer_name="뉴스 목록",
                    url_pattern="https://news.naver.com/section/{sid}",
                    selectors={"article_link": "a.sa_text_title"},
                    navigate_to_next="a.sa_text_title",
                    pagination_method="AJAX버튼"
                ),
                PageLayer(
                    layer_name="기사 상세",
                    url_pattern="https://n.news.naver.com/mnews/article/{mid}/{aid}",
                    selectors={"title": "h2.media_end_head_headline", "body": "#dic_area", "datetime": ".media_end_head_info_dateline"},
                    navigate_to_next=None
                )
            ],
            rendering_type="Dynamic CSR/JS",
            anti_bot_notes="더보기 버튼 AJAX 방식"
        )
    ]
)

# 예시 3: 구조가 다른 두 사이트 - Blueprint 2개
sample_multi_blueprint = NavigatorBlueprintCollection(
    total_jobs=2,
    blueprints=[
        NavigatorBlueprint(
            entry_urls=["https://news.naver.com/section/100"],
            total_layers=2,
            layers=[
                PageLayer(layer_name="뉴스 목록", url_pattern="https://news.naver.com/section/100",
                          selectors={"article_link": "a.sa_text_title"}, navigate_to_next="a.sa_text_title"),
                PageLayer(layer_name="기사 상세", url_pattern="https://n.news.naver.com/mnews/article/{mid}/{aid}",
                          selectors={"title": "h2.media_end_head_headline", "body": "#dic_area"}, navigate_to_next=None)
            ],
            rendering_type="Dynamic CSR/JS", anti_bot_notes="없음"
        ),
        NavigatorBlueprint(
            entry_urls=["https://www.danawa.com/product/?cateCode=112758"],
            total_layers=2,
            layers=[
                PageLayer(layer_name="상품 목록", url_pattern="https://www.danawa.com/product/?cateCode=112758",
                          selectors={"product_link": "a.prod_name"}, navigate_to_next="a.prod_name", pagination_method="URL파라미터"),
                PageLayer(layer_name="상품 상세", url_pattern="https://prod.danawa.com/info/?pcode={id}",
                          selectors={"name": "h3.prod_tit", "price": "span.low_price", "specs": "div.spec_list"}, navigate_to_next=None)
            ],
            rendering_type="Dynamic CSR/JS", anti_bot_notes="없음"
        )
    ]
)

In [4]:
print("✅ 예시 1 (단일):")
print(sample_single.model_dump_json(indent=2))
print("\n✅ 예시 2 (동일 구조, URL 2개):")
print(sample_multi_url.model_dump_json(indent=2))
print("\n✅ 예시 3 (다른 구조 사이트 2개):")
print(sample_multi_blueprint.model_dump_json(indent=2))


✅ 예시 1 (단일):
{
  "total_jobs": 1,
  "blueprints": [
    {
      "entry_urls": [
        "https://finance.naver.com/sise/sise_rise.nhn"
      ],
      "total_layers": 1,
      "layers": [
        {
          "layer_name": "주식 상승 종목 테이블",
          "url_pattern": "https://finance.naver.com/sise/sise_rise.nhn",
          "selectors": {
            "stock_name": "td.col_type1 a",
            "change_rate": "td.rate_t"
          },
          "navigate_to_next": null,
          "pagination_method": "URL파라미터"
        }
      ],
      "rendering_type": "Static SSR",
      "anti_bot_notes": "없음"
    }
  ]
}

✅ 예시 2 (동일 구조, URL 2개):
{
  "total_jobs": 1,
  "blueprints": [
    {
      "entry_urls": [
        "https://news.naver.com/section/100",
        "https://news.naver.com/section/102"
      ],
      "total_layers": 2,
      "layers": [
        {
          "layer_name": "뉴스 목록",
          "url_pattern": "https://news.naver.com/section/{sid}",
          "selectors": {
            "article_link"

# Navigator 에이전트 재설계

왜 `get_page_structure` + `browse_web`를 함께 사용하는가?

웹 데이터를 수집할 때 가장 어려운 문제는  
**"어디에서 데이터를 가져와야 하는지(구조 이해)"** 와  
**"실제로 데이터를 가져오는 것(행동 실행)"** 이 서로 다른 문제라는 점입니다.

우리는 이 두 문제를 하나의 도구로 해결하지 않고,  
**서로 역할이 다른 두 개의 도구로 분리**하여 해결합니다.

---

## 1️⃣ get_page_structure — 구조 분석 도구 (Planning Tool)

`get_page_structure`는 웹페이지의 **HTML 구조 일부를 토큰 효율적으로 반환**합니다.

이 도구의 목적은 **데이터 수집 계획(청사진)을 만드는 것**입니다.

### 역할
- 페이지의 HTML 구조 확인
- class / id 패턴 분석
- 반복되는 데이터 영역 발견 (뉴스 목록, 상품 리스트 등)
- 안정적인 CSS selector 설계

즉,
> "어디에서 데이터를 가져와야 하는가?"를 이해하는 단계입니다.

⚠️ 중요한 점:
- browser agent는 화면 의미를 이해하지만
- **재사용 가능한 CSS selector 설계에는 HTML 근거가 필요**합니다.

`get_page_structure`는 바로 이 근거를 제공합니다.

---

## 2️⃣ browse_web — 실행 도구 (Execution Tool)

`browse_web`는 browser-use 기반 에이전트를 이용해 실제 브라우저 행동을 수행합니다.

### 역할
- 클릭
- 스크롤
- 페이지 이동
- 동적 콘텐츠 로딩
- 실제 데이터 확인

즉,
> "계획한 위치에서 실제로 데이터를 가져오는 단계"입니다.

browser-use는 사람처럼 페이지를 이해하고 행동하는 데 강하지만,
기본적으로 **셀렉터 추출기(selector extractor)가 아니라 실행기(executor)** 입니다.

---

In [ ]:
import json
from dataclasses import dataclass
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langgraph.checkpoint.memory import InMemorySaver
from browser_use import Agent, Browser, ChatGoogle
from crawl4ai import AsyncWebCrawler


# ==========================================
# 공유 브라우저 (세션 유지)
# ==========================================
shared_browser = Browser(
    headless=False,
    disable_security=True,
    keep_alive=True,
)


# ==========================================
# NavigatorContext - 빈 컨텍스트 유지 (확장 대비)
# ==========================================
@dataclass
class NavigatorContext:
    pass  # 미래 확장용 (예: 로그인 정보, 프록시 설정 등)


# ==========================================
# 도구 1: get_page_structure
# crawl4ai로 HTML 구조 샘플을 토큰 효율적으로 반환
# CSS 셀렉터 탐색에 최적화
# ==========================================
@tool(parse_docstring=True)
async def get_page_structure(url: str, scraping_goal: str) -> str:
    """웹페이지 HTML을 내부 LLM이 직접 분석하여 CSS 셀렉터 결과만 반환합니다.
    Navigator는 HTML 원문을 볼 필요 없이 셀렉터 분석 결과만 받습니다.

    Args:
        url: 분석할 웹페이지 URL
        scraping_goal: 수집하려는 데이터 설명. 예) "기사 제목과 링크 URL", "상품명과 가격"
    """
    from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
    from langchain.chat_models import init_chat_model
    from langchain_core.messages import HumanMessage
    import re

    print(f"\n📐 [get_page_structure] {url}")
    print(f"   🎯 분석 목표: {scraping_goal}")

    # ── Step 1: crawl4ai로 JS 렌더링된 HTML 수집 ──
    browser_cfg = BrowserConfig(headless=True, java_script_enabled=True)
    run_cfg = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        page_timeout=15000,
        wait_for="css:main a[href]",      # 1차: '의미 있는 링크'가 등장하면 트리거
        delay_before_return_html=1.0,     # 2차: 추가 1초 대기 (JS 렌더링 완료 버퍼)
        wait_for_images=False,
    )

    try:
        async with AsyncWebCrawler(config=browser_cfg) as crawler:
            result = await crawler.arun(url=url, config=run_cfg)
    except Exception as e:
        return f"[Error] HTML 수집 실패: {e}\n→ browse_web을 사용하세요."

    # cleaned_html: class/id 보존, 광고 제거
    # 내부 LLM이 읽으므로 Navigator에게 줄 때보다 넉넉하게
    cleaned_html = (result.cleaned_html or "")[:8000]

    if not cleaned_html.strip():
        return "[Warning] HTML이 비어 있습니다. JS 렌더링 실패 가능성.\n→ browse_web을 사용하세요."

    # ── Step 2: 내부 LLM이 HTML 분석 → 셀렉터만 추출 ──
    analysis_llm = init_chat_model("google_genai:gemini-2.0-flash", temperature=0)

    analysis_prompt = f"""아래 HTML에서 "{scraping_goal}"에 해당하는 요소의 CSS 셀렉터를 찾고 JSON으로만 응답하세요.
    [분석할 HTML]
    {cleaned_html}
    [응답 형식 - JSON만, 다른 텍스트 없이]
    {{
    "selectors": {{
        "필드명": "CSS셀렉터"
    }},
    "samples": {{
        "필드명": ["실제 텍스트 예시 1", "실제 텍스트 예시 2", "실제 텍스트 예시 3"]
    }},
    "container": "목록 전체를 감싸는 컨테이너 셀렉터 (없으면 null)",
    "navigate_to_next": "다음 계층(상세 페이지)으로 이동하는 링크 셀렉터 (없으면 null)",
    "pagination": "페이지네이션 방식 (URL파라미터/AJAX버튼/무한스크롤/null)",
    "confidence": "high 또는 medium 또는 low",
    "note": "주의사항. 확인된 경우 '없음'"
    }}
    [셀렉터 작성 규칙]
    - tag + class/id 조합 필수: a.sa_text_title, div.article_box, #main_content
    - a, div, span 처럼 태그만 있는 셀렉터 절대 금지
    - HTML에 실제로 존재하는 class/id만 사용
    - 없는 값은 반드시 null로 표기하세요. 문자열 "None"은 사용 금지.
    - 텍스트와 URL을 모두 수집해야 하면 키를 분리하세요:
        예) "title": "a.sa_text_title"  (텍스트 추출용)
            "url":   "a.sa_text_title"  (href 추출용, 같은 셀렉터여도 키는 분리)
    - samples에는 HTML에서 실제로 찾은 텍스트를 기재하세요
    - confidence low면 note에 근거 명시
    """
    response = await analysis_llm.ainvoke([HumanMessage(analysis_prompt)])
    raw = response.content.strip()

    # 마크다운 코드블록 제거 (```json ... ``` 대응)
    json_match = re.search(r'\{.*\}', raw, re.DOTALL)
    if json_match:
        try:
            parsed = json.loads(json_match.group())
            print(f"   ✅ 셀렉터 추출: {parsed.get('selectors')} / confidence={parsed.get('confidence')}")
            return json.dumps(parsed, ensure_ascii=False, indent=2)
        except json.JSONDecodeError:
            pass

    # JSON 파싱 실패 시 원문 반환
    print(f"   ⚠️ JSON 파싱 실패, 원문 반환")
    return raw


# ==========================================
# 도구 2: browse_web
# 동적 인터랙션이 필요할 때만 사용 (클릭, 스크롤 등)
# 토큰 비용이 크므로 get_page_structure로 해결 안 될 때만 호출
# ==========================================
@tool(parse_docstring=True)
async def browse_web(url: str, instruction: str) -> str:
    """실제 브라우저로 웹페이지를 방문하여 동적 인터랙션을 수행합니다.
    
    사용 시점:
    - 클릭, 스크롤, 검색어 입력, 로그인 등 인터랙션이 필요한 경우
    - get_page_structure가 "[Warning]" 또는 "[Error]"를 반환한 경우 (폴백)
    - 현재 페이지에서 이어서 대화형으로 작업하는 경우
    
    Args:
        url: 이동할 URL. 현재 페이지에서 이어서 작업하려면 빈 문자열("")을 전달하세요.
        instruction: 수행할 구체적인 작업. 원하는 결과물을 명확히 기술하세요.
                     예) "검색창에 '무선 마우스'를 입력하고 결과 목록의 상위 3개 상품명을 반환하세요"
                     예) "4번째 상품의 이름과 가격을 알려주세요"
                     예) "기사 목록의 CSS 셀렉터를 찾아주세요 (get_page_structure 폴백)"
    """
    print(f"\n🌐 [browse_web] {'→ ' + url if url else '현재 페이지 이어서'}")
    print(f"   📋 작업: {instruction}")
    bu_llm = ChatGoogle(model="gemini-2.0-flash")
    # url 유무에 따라 분기
    if url:
        nav_prefix = (
            f"첫 번째 액션으로 반드시 navigate를 실행하여 아래 URL로 이동하세요.\n"
            f"현재 브라우저 상태와 관계없이 즉시 이동부터 시작합니다.\n\n"
            f"[이동할 URL]\n{url}\n\n"
            f"[이동 후 수행할 작업]"
        )
    else:
        nav_prefix = (
            f"현재 열려있는 페이지에서 바로 아래 작업을 수행하세요.\n"
            f"navigate 액션으로 다른 페이지로 이동하지 마세요.\n\n"
            f"[수행할 작업]"
        )
    task = f"""{nav_prefix}
    {instruction}
    [결과 보고 규칙]
    - 작업 결과를 구체적으로 보고하세요.
    - 요소를 find_elements로 찾기 어렵다면, screenshot으로 화면을 직접 확인한 뒤
    시각적으로 목표 요소를 파악하세요.
    - CSS 셀렉터가 필요한 작업이라면:
        tag + class/id 조합으로 작성하세요. 예) "a.sa_text_title", "#dic_area"
        "a", "div" 처럼 태그만 있는 셀렉터는 사용 금지.
    - 확인 불가능한 정보는 "확인 불가"로 명시하세요.
    - 작업 완료 후 현재 페이지 URL과 상태를 함께 보고하세요.
    """
    agent = Agent(task=task, llm=bu_llm, use_vision="auto", browser=shared_browser)
    history = await agent.run(max_steps=15)
    result = history.final_result() or "탐색 완료, 결과 반환 없음"
    print(f"\n✅ [browse_web 완료] {result[:200]}...")
    return result

In [17]:

# ==========================================
# Navigator 에이전트
# ==========================================
nav_model = init_chat_model("google_genai:gemini-flash-latest", temperature=0.1)
nav_checkpointer = InMemorySaver()

NAVIGATOR_SYSTEM_PROMPT = """
당신은 웹 크롤링 파이프라인의 총괄 설계자 'Navigator'입니다.
도구를 상황에 맞게 판단하여 사용하고,
최종적으로 Coder가 즉시 실행할 수 있는 안정적인 크롤링 Blueprint를 설계합니다.

당신의 역할은 도구를 기계적으로 실행하는 것이 아니라,
웹 구조를 이해하고 데이터 수집 전략(청사진)을 설계하는 것입니다.

────────────────
[도구 역할]

■ browse_web(url, instruction)
탐색, 인터랙션, 그리고 검증(Validation)을 위한 도구입니다.

다음 상황에서 사용하세요:
- 원하는 데이터가 존재하는 URL을 파악해야 할 때
- 클릭, 검색 입력, 스크롤 등 실제 조작이 필요할 때
- JS 렌더링 이후 콘텐츠 확인이 필요할 때
- get_page_structure의 samples만으로 검증이 불충분할 때

url이 ""이면 현재 페이지에서 작업을 이어갑니다.

---

■ get_page_structure(url, scraping_goal)
데이터 추출을 위한 CSS 셀렉터 구조를 분석합니다.

사용 조건:
- 데이터가 존재하는 URL이 확정된 이후 호출합니다.
- HTML 구조 기반 selector 후보를 생성하는 단계입니다.
- 반환 JSON의 selectors를 그대로 Blueprint에 사용합니다.

반환 형식:
{
  "selectors": {"필드명": "CSS셀렉터"},
  "samples":   {"필드명": ["실제 텍스트 예시1", "예시2", "예시3"]},
  "container": "컨테이너 셀렉터",
  "navigate_to_next": "다음 계층 링크 셀렉터",
  "pagination": "페이지네이션 방식",
  "confidence": "high/medium/low",
  "note": "주의사항"
}

samples를 먼저 확인하여 셀렉터가 올바른 요소를 가리키는지 1차 검증하세요.
※ 당신은 HTML 원문을 직접 보지 않습니다.

────────────────
[핵심 설계 원칙]

1. browse_web는 "탐색 도구"이면서 동시에 "검증 도구"입니다.
2. get_page_structure는 구조(structure)를 찾고,
   samples로 1차 검증 → 부족하면 browse_web으로 2차 검증합니다.
3. 항상 가장 효율적인 도구를 판단하여 선택하세요.
   (항상 먼저 사용해야 하는 도구는 없습니다.)

────────────────
[작업 흐름]

1. 지시를 받으면 수집 목표와 초기 URL을 파악합니다.

2. 목표 URL 상태 판단:
   - 데이터 위치가 명확 → get_page_structure 호출
   - 위치가 불명확 → browse_web으로 탐색

3. get_page_structure 결과 분석:
   - confidence = high/medium
        → samples를 확인합니다.
          samples가 수집 목표와 일치 → Blueprint 후보로 채택
          samples가 불명확       → browse_web으로 실제 화면과 대조 검증
   - confidence = low 또는 Warning/Error
        → browse_web으로 구조 재확인 후 재분석

4. 상세 페이지가 존재하면:
   navigate_to_next 링크 중 하나를 샘플로 선택하여
   get_page_structure를 재호출합니다.

5. 구조와 의미가 모두 확인되면 Blueprint를 완성합니다.

────────────────
[Blueprint 작성 규칙]

- get_page_structure JSON의 selectors를 그대로 사용합니다.
- navigate_to_next가 있으면 layers[i].navigate_to_next에 기록합니다.
- 구조가 동일하고 entry URL만 다른 경우 Blueprint를 통합합니다.
- 구조가 근본적으로 다르면 Blueprint를 분리합니다.

────────────────
[중요 행동 규칙]

- selector는 반드시 실제 데이터 요소와 대응되어야 합니다.
- 숨겨진 요소, 광고 영역, 템플릿 영역 선택을 피하세요.
- 검증 순서: samples 확인(무료) → browse_web 검증(비용 발생) 순으로 진행하세요.
"""


navigator_agent = create_agent(
    model=nav_model,
    system_prompt=NAVIGATOR_SYSTEM_PROMPT,
    context_schema=NavigatorContext,   # 빈 컨텍스트 유지 (확장 대비)
    tools=[get_page_structure, browse_web],
    checkpointer=nav_checkpointer,
    response_format=ToolStrategy(NavigatorBlueprintCollection),
)

print("✅ Navigator 에이전트 준비 완료")
print("   - 도구 1: get_page_structure (crawl4ai, 토큰 효율적)")
print("   - 도구 2: browse_web (browser-use, 인터랙션 전용)")
print("   - 출력: NavigatorBlueprintCollection")

✅ Navigator 에이전트 준비 완료
   - 도구 1: get_page_structure (crawl4ai, 토큰 효율적)
   - 도구 2: browse_web (browser-use, 인터랙션 전용)
   - 출력: NavigatorBlueprintCollection


### 네비게이터 테스트

In [18]:
import json, os

ARTIFACT_DIR = os.path.join(project_root, "code_artifacts")
os.makedirs(ARTIFACT_DIR, exist_ok=True)

def save_blueprints(collection: NavigatorBlueprintCollection, prefix: str):
    """NavigatorBlueprintCollection을 Blueprint 개수만큼 별도 파일로 저장"""
    saved_paths = []
    for i, bp in enumerate(collection.blueprints):
        filename = f"blueprint_{prefix}_{i+1}.json"
        filepath = os.path.join(ARTIFACT_DIR, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(bp.model_dump(), f, ensure_ascii=False, indent=2)
        saved_paths.append(filepath)
        print(f"  💾 저장 완료: {filepath}")
    return saved_paths

print(f"✅ 유틸리티 준비 완료. 아티팩트 경로: {ARTIFACT_DIR}")


✅ 유틸리티 준비 완료. 아티팩트 경로: /mnt/d/agent_research/github-repos/home/AAWS_project/code_artifacts


In [19]:
from langchain_core.messages import HumanMessage

# ──────────────────────────────────────────────────────
# 테스트 (1): 정치 섹션 기사 제목 5개 수집
# ──────────────────────────────────────────────────────
print("=" * 60)
print("🧪 테스트 (1): 정치 섹션 기사 제목 5개 수집")
print("=" * 60)

nav_config_1 = {"configurable": {"thread_id": "nav_test_1"}}
nav_context_1 = NavigatorContext()  # 빈 컨텍스트

response_1 = await navigator_agent.ainvoke(
    {"messages": [HumanMessage(
        "https://news.naver.com에서 정치 섹션의"
        "최신 기사 제목 5개와 각 기사 URL을 수집하는 Blueprint를 작성해주세요. "
        "탐색은 이 URL에서 시작하세요."
    )]},
    config=nav_config_1,
    context=nav_context_1,
)

collection_1: NavigatorBlueprintCollection = response_1["structured_response"]

print(f"\n✅ 결과: {collection_1.total_jobs}개 Blueprint 생성")
for i, bp in enumerate(collection_1.blueprints):
    print(f"\n  [Blueprint {i+1}]")
    print(f"    entry_urls : {bp.entry_urls}")
    print(f"    계층 수    : {bp.total_layers}")
    for j, layer in enumerate(bp.layers):
        print(f"    Layer {j+1}   : {layer.layer_name} | selectors={layer.selectors}")

print("\n📁 파일 저장:")
paths_1 = save_blueprints(collection_1, prefix="test1")

🧪 테스트 (1): 정치 섹션 기사 제목 5개 수집


INFO     [Agent] 🎯 Task: 첫 번째 액션으로 반드시 navigate를 실행하여 아래 URL로 이동하세요.
현재 브라우저 상태와 관계없이 즉시 이동부터 시작합니다.

[이동할 URL]
https://news.naver.com

[이동 후 수행할 작업]
    '정치' 섹션의 링크를 찾아서 해당 페이지로 이동하세요.
    [결과 보고 규칙]
    - 작업 결과를 구체적으로 보고하세요.
    - 요소를 find_elements로 찾기 어렵다면, screenshot으로 화면을 직접 확인한 뒤
    시각적으로 목표 요소를 파악하세요.
    - CSS 셀렉터가 필요한 작업이라면:
        tag + class/id 조합으로 작성하세요. 예) "a.sa_text_title", "#dic_area"
        "a", "div" 처럼 태그만 있는 셀렉터는 사용 금지.
    - 확인 불가능한 정보는 "확인 불가"로 명시하세요.
    - 작업 완료 후 현재 페이지 URL과 상태를 함께 보고하세요.
    
INFO     [Agent] 📦 Newer version available: 0.11.13 (current: 0.11.12). Upgrade with: uv add browser-use==0.11.13
INFO     [Agent] Starting a browser-use agent with version 0.11.12, with provider=google and model=gemini-2.0-flash



🌐 [browse_web] → https://news.naver.com
   📋 작업: '정치' 섹션의 링크를 찾아서 해당 페이지로 이동하세요.


INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: Agent initialized. No previous action to evaluate.
INFO     [Agent]   🧠 Memory: No previous actions. Navigating to the specified URL.
INFO     [Agent]   🎯 Next goal: Navigate to https://news.naver.com
INFO     [Agent]   ▶️   navigate: url: https://news.naver.com, new_tab: False
INFO     [tools] 🔗 Navigated to https://news.naver.com
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully navigated to the specified URL. Verdict: Success
INFO     [Agent]   🧠 Memory: Navigated to the 네이버 뉴스 main page. Now I need to find the '정치' section link and navigate to it.
INFO     [Agent]   🎯 Next goal: Find the '정치' section link and navigate to it.
INFO     [Agent]   ▶️   find_elements: selector: a[role='menuitem'], attributes: ['href', 'text'], max_results: 100, include_text: True
INFO     [tools] 🔍 Found 14 elements matching "a[role='menuitem']".
INFO     [Agent] 📋 Plan updated with 0 steps
INFO 


✅ [browse_web 완료] 현재 페이지 URL: https://news.naver.com/section/100
페이지 상태: '정치' 섹션 페이지에 성공적으로 접속했습니다. 다양한 정치 관련 기사 및 메뉴(언론사별, 정치, 경제, 사회, 생활/문화, IT/과학, 세계, 랭킹, 신문보기, 오피니언, TV, 팩트체크, 알고리즘 안내, 정정보도 모음, MY)를 확인할 수 있습니다....

📐 [get_page_structure] https://news.naver.com/section/100
   🎯 분석 목표: 기사 제목과 기사 상세 페이지 링크 URL


...


✅ [browse_web 완료] CSS 셀렉터를 찾을 수 없습니다. 페이지 구조가 예상과 다릅니다. Chrome DevTools를 사용하여 HTML 구조를 분석하고 CSS 셀렉터를 식별해야 합니다. 현재 페이지 URL: https://news.naver.com/section/100...


INFO     [Agent] 🎯 Task: 첫 번째 액션으로 반드시 navigate를 실행하여 아래 URL로 이동하세요.
현재 브라우저 상태와 관계없이 즉시 이동부터 시작합니다.

[이동할 URL]
https://news.naver.com/section/100

[이동 후 수행할 작업]
    '.sa_text_title' 또는 '.sa_item' 클래스가 페이지 내에 있는지 확인하고, 있으면 해당 태그들의 텍스트 5개를 추출해 주세요.
    [결과 보고 규칙]
    - 작업 결과를 구체적으로 보고하세요.
    - 요소를 find_elements로 찾기 어렵다면, screenshot으로 화면을 직접 확인한 뒤
    시각적으로 목표 요소를 파악하세요.
    - CSS 셀렉터가 필요한 작업이라면:
        tag + class/id 조합으로 작성하세요. 예) "a.sa_text_title", "#dic_area"
        "a", "div" 처럼 태그만 있는 셀렉터는 사용 금지.
    - 확인 불가능한 정보는 "확인 불가"로 명시하세요.
    - 작업 완료 후 현재 페이지 URL과 상태를 함께 보고하세요.
    
INFO     [Agent] 📦 Newer version available: 0.11.13 (current: 0.11.12). Upgrade with: uv add browser-use==0.11.13
INFO     [Agent] Starting a browser-use agent with version 0.11.12, with provider=google and model=gemini-2.0-flash



🌐 [browse_web] → https://news.naver.com/section/100
   📋 작업: '.sa_text_title' 또는 '.sa_item' 클래스가 페이지 내에 있는지 확인하고, 있으면 해당 태그들의 텍스트 5개를 추출해 주세요.


INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: Agent initialized. No previous action to evaluate.
INFO     [Agent]   🧠 Memory: No previous actions or information stored.
INFO     [Agent]   🎯 Next goal: Navigate to the specified URL: https://news.naver.com/section/100.
INFO     [Agent]   ▶️   navigate: url: https://news.naver.com/section/100, new_tab: False
INFO     [tools] 🔗 Navigated to https://news.naver.com/section/100
INFO     [Agent] 📋 Plan updated with 0 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully navigated to the specified URL. Verdict: Success
INFO     [Agent]   🧠 Memory: Navigated to https://news.naver.com/section/100. Need to find '.sa_text_title' or '.sa_item' elements and extract the first 5 texts.
INFO     [Agent]   🎯 Next goal: Find elements with class '.sa_text_title' or '.sa_item' and extract the text from the first 5 elements.
INFO     [Agent]   ▶️  [1/2] find_elements: selector: .sa_text_title, 


✅ [browse_web 완료] The text from the first 5 elements with class '.sa_text_title' are:

1. 당조직 흡수 거부한 공취모…‘친청 vs 반청’ 대립 계속될 듯
2. 한경협 “개정 상법개정안 존중…특정 목적 자사주는 추후 논의해야”
3. 민주 김용민 "법 왜곡죄는 누더기...지도부 책임져야"
4. 美민주, '생활비부담' 부각하...

✅ 결과: 1개 Blueprint 생성

  [Blueprint 1]
    entry_urls : ['https://news.naver.com/section/100']
    계층 수    : 1
    Layer 1   : 정치 섹션 기사 목록 | selectors={'기사 제목': 'a.sa_text_title', '기사 URL': 'a.sa_text_title'}

📁 파일 저장:
  💾 저장 완료: /mnt/d/agent_research/github-repos/home/AAWS_project/code_artifacts/blueprint_test1_1.json


# Coder 에이전트 재설계

In [21]:
import os
import subprocess
from dataclasses import dataclass
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import FilesystemFileSearchMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# ==========================================
# Coder 작업 공간 설정
# ==========================================
ARTIFACT_DIR = os.path.join(project_root, "code_artifacts")
os.makedirs(ARTIFACT_DIR, exist_ok=True)
print(f"✅ 코더 작업 공간: {ARTIFACT_DIR}")

# ==========================================
# Coder Context - 작업 환경 설정 (경로 정보)
# ==========================================
@dataclass
class CoderContext:
    artifact_dir: str   # 코드/결과 저장 디렉토리
    blueprint_path: str # Navigator가 저장한 Blueprint JSON 경로



# ==========================================
# Coder 도구: 파이썬 코드 실행 + 디버깅
# ==========================================
@tool(parse_docstring=True)
def execute_python_code(code: str, filename: str = "scraper.py") -> str:
    """주어진 파이썬 코드를 파일로 저장하고 실행한 뒤 결과(출력/에러)를 반환합니다.
    코드 테스트 및 디버깅에 사용합니다.

    Args:
        code: 실행할 완전한 파이썬 스크립트 (모든 import 포함 필수).
        filename: 저장할 파일명 (기본값: 'scraper.py').
    """
    safe_filename = os.path.basename(filename)
    filepath = os.path.join(ARTIFACT_DIR, safe_filename)
    print(f"\n🐍 [Coder Tool] '{filepath}' 저장 및 실행 중...")

    try:
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(code)

        result = subprocess.run(
            ["python", safe_filename],
            cwd=ARTIFACT_DIR,
            capture_output=True,
            text=True,
            timeout=60  # 스크래핑은 시간이 걸릴 수 있음
        )

        output = result.stdout
        if result.stderr:
            output += f"\n[Error Output]\n{result.stderr}"
        if not output.strip():
            output = "[System] 실행 완료 (출력 없음)"

        return output

    except subprocess.TimeoutExpired:
        return "[Error] 실행 시간(60초) 초과. 타임아웃 설정을 확인하세요."
    except Exception as e:
        return f"[System Error] 실행 오류: {str(e)}"


# ==========================================
# Coder 에이전트 조립
# ==========================================
coder_model = init_chat_model("google_genai:gemini-2.0-flash", temperature=0.1)
coder_checkpointer = InMemorySaver()

CODER_SYSTEM_PROMPT = """
당신은 웹 크롤링 전문 'Coder(개발자)' 에이전트입니다.
Navigator가 전달한 Blueprint를 기반으로, 실행 가능한 파이썬 크롤링 코드를 작성하고 직접 실행합니다.

[작업 환경 (Context)]
- artifact_dir: 코드와 결과 파일을 저장하는 작업 디렉토리
- blueprint_path: 참조할 Blueprint JSON 파일 경로

[Blueprint 해석 방법]
Blueprint의 `layers` 리스트는 탐색 순서대로 정렬되어 있습니다.
- `entry_urls`: 크롤링 시작 URL 목록 (여러 개면 각각 반복)
- `layers[0]`: 최초 진입 URL에서 첫 번째 데이터 수집
- `layers[i].navigate_to_next` 값이 None이 아니면 → 그 셀렉터로 링크 추출 후 다음 계층으로 이동
- `layers[-1]` (마지막): `navigate_to_next=None`, 최종 데이터 수집

[selectors 해석 방법]
Blueprint의 selectors는 {필드명: CSS셀렉터} 딕셔너리입니다.
- 같은 셀렉터가 title과 url에 모두 있으면:
    title → 해당 요소의 .text (또는 .inner_text())
    url   → 해당 요소의 href 속성 (또는 .get_attribute('href'))
- 예) selectors = {"title": "a.sa_text_title", "url": "a.sa_text_title"}
    → elements = page.query_selector_all("a.sa_text_title")
    → title = el.inner_text()
    → url = el.get_attribute("href")

[코드 작성 규칙]
1. `rendering_type`에 따라 라이브러리 선택:
   - "Static SSR"    → `requests` + `BeautifulSoup`
   - "Dynamic CSR/JS" → `playwright` (동기 방식: sync_playwright)
2. Playwright 사용 시 반드시 동기(sync) 방식으로 작성하세요:
   from playwright.sync_api import sync_playwright
   with sync_playwright() as p:
       browser = p.chromium.launch(headless=True)
       page = browser.new_page()
       page.goto(url)
       page.wait_for_load_state("networkidle")
3. `layers` 리스트를 인덱스 순서대로 처리하는 함수를 작성하세요.
4. Blueprint의 selectors를 그대로 사용하세요. 추측으로 변경 금지.
5. 결과는 JSON 파일로 저장하세요 (파일명: scraping_result.json).
6. 코드 작성 후 반드시 `execute_python_code`로 실행하여 결과를 검증하세요.
7. 에러 발생 시 로그를 분석하여 코드를 수정한 뒤 다시 실행하세요. 성공할 때까지 반복합니다.
"""


coder_agent = create_agent(
    model=coder_model,
    system_prompt=CODER_SYSTEM_PROMPT,
    context_schema=CoderContext,
    tools=[execute_python_code],
    middleware=[
        FilesystemFileSearchMiddleware(
            root_path=ARTIFACT_DIR,
            use_ripgrep=True,
            max_file_size_mb=10,
        )
    ],
    checkpointer=coder_checkpointer
)

print("✅ Coder 에이전트 준비 완료")

✅ 코더 작업 공간: /mnt/d/agent_research/github-repos/home/AAWS_project/code_artifacts
✅ Coder 에이전트 준비 완료


In [23]:
SCRAPING_GOAL

NameError: name 'SCRAPING_GOAL' is not defined

In [24]:
from langchain_core.messages import HumanMessage
import json

SCRAPING_GOAL = "정치 섹션의 최신 기사 제목과 URL 5개 수집"

# blueprint_dict가 없으면 저장된 파일에서 로드
if "blueprint_dict" not in dir():
    blueprint_path = os.path.join(ARTIFACT_DIR, "blueprint_test1_1.json")
    with open(blueprint_path, "r", encoding="utf-8") as f:
        blueprint_dict = json.load(f)
    print(f"📂 Blueprint 파일에서 로드: {blueprint_path}")
else:
    print("✅ 기존 blueprint_dict 사용")

# paths_1이 없으면 blueprint_path 직접 지정
if "paths_1" not in dir():
    paths_1 = [os.path.join(ARTIFACT_DIR, "blueprint_test1_1.json")]

# ──────────────────────────────────────────────
# Coder 실행
# ──────────────────────────────────────────────
coder_config = {"configurable": {"thread_id": "coder_session"}}
coder_context = CoderContext(
    artifact_dir=ARTIFACT_DIR,
    blueprint_path=paths_1[0],
)

coder_response = await coder_agent.ainvoke(
    {"messages": [HumanMessage(
        f"아래 Blueprint를 기반으로 '{SCRAPING_GOAL}' 크롤링 코드를 작성하고 직접 실행해주세요.\n\n"
        f"[Blueprint]\n"
        f"{json.dumps(blueprint_dict, ensure_ascii=False, indent=2)}\n\n"
        f"결과는 artifact_dir에 'scraping_result.json'으로 저장하세요."
    )]},
    config=coder_config,
    context=coder_context,
)

print(f"\n✅ Coder 완료!")
print(f"\n🤖 Coder 최종 리포트:\n{coder_response['messages'][-1].content}")

📂 Blueprint 파일에서 로드: /mnt/d/agent_research/github-repos/home/AAWS_project/code_artifacts/blueprint_test1_1.json

✅ Coder 완료!

🤖 Coder 최종 리포트:
```python
from playwright.sync_api import sync_playwright
import json

def scrape_news(entry_urls, layers, artifact_dir):
    results = []
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        for entry_url in entry_urls:
            page.goto(entry_url)
            page.wait_for_load_state("networkidle")

            layer = layers[0]
            selectors = layer["selectors"]
            title_selector = selectors["기사 제목"]
            url_selector = selectors["기사 URL"]

            elements = page.query_selector_all(title_selector)

            for i, el in enumerate(elements):
                if i >= 5:
                    break

                title = el.inner_text()
                url = el.get_attribute("href")

                results.append({"기사 제목": title, "기사 URL"

In [25]:
for m in coder_response['messages']:
    m.pretty_print()

================================ Human Message =================================

아래 Blueprint를 기반으로 '정치 섹션의 최신 기사 제목과 URL 5개 수집' 크롤링 코드를 작성하고 직접 실행해주세요.

[Blueprint]
{
  "entry_urls": [
    "https://news.naver.com/section/100"
  ],
  "total_layers": 1,
  "layers": [
    {
      "layer_name": "정치 섹션 기사 목록",
      "url_pattern": "https://news.naver.com/section/100",
      "selectors": {
        "기사 제목": "a.sa_text_title",
        "기사 URL": "a.sa_text_title"
      },
      "navigate_to_next": "None",
      "pagination_method": "None"
    }
  ],
  "rendering_type": "Dynamic CSR/JS",
  "anti_bot_notes": "없음 (단, 짧은 시간 내 대량 요청 시 캡차 발생 가능)"
}

결과는 artifact_dir에 'scraping_result.json'으로 저장하세요.
================================== Ai Message ==================================

```python
from playwright.sync_api import sync_playwright
import json

def scrape_news(entry_urls, layers, artifact_dir):
    results = []
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)


In [29]:
coder_response = await coder_agent.ainvoke(
    {"messages": [HumanMessage("결과 파일을 보여줘")]},
    config=coder_config,
    context=coder_context)

for m in coder_response['messages']:
    m.pretty_print()


🐍 [Coder Tool] '/mnt/d/agent_research/github-repos/home/AAWS_project/code_artifacts/scraper.py' 저장 및 실행 중...

🐍 [Coder Tool] '/mnt/d/agent_research/github-repos/home/AAWS_project/code_artifacts/scraper.py' 저장 및 실행 중...

🐍 [Coder Tool] '/mnt/d/agent_research/github-repos/home/AAWS_project/code_artifacts/scraper.py' 저장 및 실행 중...


WARNING  [langchain_google_genai.chat_models] Gemini produced an empty response. Continuing with empty message
Feedback: None


In [27]:
await shared_browser.stop() # 이제 browser를 닫습니다.

INFO     [BrowserSession] ✅ Browser session reset complete


### 성공!

```
HumanMessage (URL + 목표)
    ↓
Navigator 에이전트
    ├─ get_page_structure() → confidence=low (JS 렌더링 실패)
    └─ browse_web() → a.sa_text_title 확인
    ↓
NavigatorBlueprintCollection
    { selectors: {"기사 제목": "a.sa_text_title", ...} }
    ↓
Coder 에이전트
    ├─ Playwright 코드 작성
    └─ execute_python_code() 실행
    ↓
scraping_result.json ✅
```

```
[
    {
        "기사 제목": "與 '尹정권 조작기소 국조 추진위' 설치하자 '李 공취모' 탈퇴 행렬",
        "기사 URL": "https://n.news.naver.com/mnews/article/656/0000168370"
    },
    {
        "기사 제목": "강기정 광주시장 \"잠시만 광주공항에서 국제선을 운영하겠다\"",
        "기사 URL": "https://n.news.naver.com/mnews/article/421/0008793751"
    },
    {
        "기사 제목": "與, 법왜곡죄법 수정키로…\"명확성 추가해 위헌 소지 최소화\"",
        "기사 URL": "https://n.news.naver.com/mnews/article/001/0015924416"
    },
    {
        "기사 제목": "'자사주 의무소각' 3차 상법 개정안 국회 본회의 통과(상보)",
        "기사 URL": "https://n.news.naver.com/mnews/article/018/0006224624"
    },
    {
        "기사 제목": "TK 통합 불발에 국힘 ‘텃밭’ 영남권 의원들도 뿔났다",
        "기사 URL": "https://n.news.naver.com/mnews/article/005/0001833933"
    }
]


```

# 🏆 실습: LangGraph로 파이프라인 자동화하기

이번 실습에서 우리는 Navigator와 Coder를 **별도로 실행**하고 중간에 직접 Blueprint를 넘겨주었습니다.

```python
# 지금까지의 방식: 수동 연결
nav_response = await navigator_agent.ainvoke(...)   # 1단계 직접 실행
blueprint = nav_response["structured_response"]      # 중간 결과 수동 추출
coder_response = await coder_agent.ainvoke(...)      # 2단계 직접 실행
```


이 방식은 간단하지만, 한계가 있습니다.

- 사람이 중간에 개입해야 다음 단계가 실행됩니다.
- 새로운 에이전트가 추가되면 연결 코드가 계속 늘어납니다.
- 조건에 따른 분기(예: 실패 시 재시도)를 구현하기가 복잡합니다.


## 목표

LangGraph StateGraph를 사용하여 Navigator → Coder 파이프라인을 자동화하세요.
사용자가 수집 목표만 입력하면, 아래 흐름이 자동으로 실행되어야 합니다:

```
[START]
   ↓
[Navigator 노드]  - 목표 URL 탐색 → Blueprint 생성
   ↓
[Coder 노드]      - Blueprint 해석 → 코드 작성 → 실행
   ↓
[END]
```